In [1]:
import os

In [2]:
%pwd

'e:\\work\\DS\\Wine-Quality-Prediction\\research'

In [3]:
os.chdir("e:/work/DS/Wine-Quality-Prediction/")

In [4]:
%pwd

'e:\\work\\DS\\Wine-Quality-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.Wine_Quality_Prediction.constants import *
from src.Wine_Quality_Prediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):  
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir,
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.Wine_Quality_Prediction import logger
from src.Wine_Quality_Prediction.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file,
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-21 22:13:12,372]: INFO: yaml file: config\config.yaml loaded successfully
[2025-12-21 22:13:12,375]: INFO: yaml file: params.yaml loaded successfully
[2025-12-21 22:13:12,377]: INFO: yaml file: schema.yaml loaded successfully
[2025-12-21 22:13:12,379]: INFO: created directory at: artifacts
[2025-12-21 22:13:12,380]: INFO: created directory at: artifacts/data_ingestion
[2025-12-21 22:13:13,819]: INFO: artifacts/data_ingestion/winequality-red.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "304b279ece9075582ea58e0efe9a417da05d7ec3f34a88eff32cb9bddc8d0453"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A58B:120DEC:34E2E1:6D3790:69480E17
Accept-Ranges: bytes
Date: Sun, 21 Dec 2025 15:11:19 GMT
Via: 1.1 varnish
